In [1]:
import numpy as np
import pandas as pd
from importlib import reload
import heston
import levenberg_marquardt
reload(levenberg_marquardt)
reload(heston)
from heston import fHes, JacHes, MarketParameters, ModelParameters
from levenberg_marquardt import Levenberg_Marquardt
from typing import Tuple

In [ ]:
df = pd.read_csv(f"../datasets/deribit_options_chain_2022-12-01_OPTIONS.csv")


In [ ]:
def select_data(df: pd.DataFrame, currency: str  = "BTC"):
    # take single currency
    df = df.drop(columns = ["exchange", "open_interest", 
                    "last_price", "bid_price", "bid_amount", "bid_iv",
                           "ask_price", "ask_amount", "ask_iv", "underlying_index",
                           "delta", "gamma", "vega", "theta" ,"rho"])
    ind = [symbol.startswith(currency) for symbol in df.symbol]
    data = df[ind].reset_index(drop=True)
    #tau is time before expiration in years
    data['tau'] = (data.expiration - data.timestamp) / 1e6 / 3600 / 24 / 365
    K = data.strike_price.values
    F = data.underlying_price.values

    payoff = np.maximum(F - K, 0.0)
    payoff[data.type == 'put'] = np.maximum( K - F, 0.0)[data.type == 'put']
    data["payoff"] = payoff
    
    #inverse 
    data["inverse_payoff"] =  data["payoff"]/data["underlying_price"]
    
    data_grouped = data.groupby(['type', 'expiration', 'strike_price'])\
    .agg(lambda x: x.iloc[-1]).reset_index().drop(["timestamp"], axis=1)
    data_grouped = data_grouped[data_grouped["tau"]>0.0]
    # Only out of the money
    data_grouped = data_grouped[
        ((data_grouped["type"] == "call") & 
        (data_grouped["underlying_price"] <= data_grouped["strike_price"])) | 
       ((data_grouped["type"] == "put") &
        (data_grouped["underlying_price"] >= data_grouped["strike_price"])) 
    ]
    data_grouped["mark_price_usd"] = data_grouped["mark_price"] * data_grouped["underlying_price"]
    
    return data_grouped
    
    

In [ ]:
sample = select_data(df, currency = "ETH")

In [ ]:
karr = sample.strike_price.to_numpy(dtype=np.float64)
carr = sample.mark_price_usd.to_numpy(dtype=np.float64)
tarr = sample.tau.to_numpy(dtype=np.float64)
r_val = np.float64(0.0)
S_val = np.float64(sample.underlying_price.mean())

In [ ]:
market = MarketParameters(K=karr, T=tarr, S=S_val, r=r_val, C = carr)


def proj_heston( heston_params : np.ndarray )->np.ndarray:
    """
        This funciton project heston parameters into valid range
        Attributes:
            heston_params(np.ndarray): model parameters
        
        Returns:
            heston_params(np.ndarray): clipped parameters
    """
    eps = 1e-4
    for i in range(len(heston_params) // 5):
        a, b, c, rho, v0 = heston_params[i * 5 : i * 5 + 5]
#         v0, theta, rho, k, sig = heston_params[i * 5 : i * 5 + 5]
        a = np.clip(a, eps, 100.0)
        b = np.clip(b, eps, 100.0)
        c = np.clip(c, eps, 100.0)
        rho = np.clip(rho, -1 + eps, 1 - eps)
        v0 = np.clip(v0, eps, 100.0)
        heston_params[i * 5 : i * 5 + 5] = a, b, c, rho, v0
#         heston_params[i * 5 : i * 5 + 5] = v0, theta, rho, k, sig
    
    return heston_params

def get_residuals( heston_params:np.ndarray ) -> Tuple[ np.ndarray, np.ndarray ]:
    '''
        This function calculates residuals and Jacobian matrix
        Args:
            heston_params(np.ndarray): model params
        Returns:
            res(np.ndarray) : vector or residuals
            J(np.ndarray)   : Jacobian
    '''
    # needed format to go
    model_parameters = ModelParameters(
            heston_params[0],
            heston_params[1],
            heston_params[2],
            heston_params[3],
            heston_params[4])
    # тут ок в целом, надо подогнать дальше и смотреть
    #  чтоб ваще те параметры подставлялись в якобиан
    C = fHes(
    model_parameters=model_parameters,
    market_parameters=market,
    )

    J = JacHes(
    model_parameters=model_parameters, 
    market_parameters=market)

    K = karr
    F = np.ones(len(K))*market.S
    weights = np.ones_like(K)
    weights = weights / np.sum(weights)
    typ = True
    P = C + np.exp(-market.r * market.T) * ( K - F )
    X_ = C
    X_[~typ] = P[~typ]
    res = X_ - market.C
    return res * weights,  J @ np.diag(weights)


a = np.float64(5.0)  # kappa                     |  mean reversion rate
b = np.float64(0.2)  # v_infinity               |  long term variance
c = np.float64(7.0)  # sigma                    |  variance of volatility
rho = np.float64(0.3)  # rho                    |  correlation between spot and volatility
v0 = np.float64(0.1) # init variance            | initial variance   

start_params = np.array([a, b, c, rho, v0])
res = Levenberg_Marquardt(100, get_residuals, proj_heston, start_params)
calibrated_params = np.array(res["x"], dtype = np.float64)
calibrated_params



In [ ]:
res

In [ ]:
calibrated_params_model = ModelParameters(
            calibrated_params[0],
            calibrated_params[1],
            calibrated_params[2],
            calibrated_params[3],
            calibrated_params[4])
    # тут ок в целом, надо подогнать дальше и смотреть
    #  чтоб ваще те параметры подставлялись в якобиан
calibr_C = fHes(
    model_parameters=calibrated_params_model,
    market_parameters=market,
    )
sample["mark_price_calibrated"] = calibr_C
sample